<a href="https://colab.research.google.com/github/CaioHenrique28/TCC_AMAN_osint/blob/main/1_coleta_massiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TCC: Análise de Sentimentos - Exército Brasileiro (NLP)**
**Autor:** Cadete Caio Henrique
**Módulo 1:** Coleta de Dados Automatizada (ETL)

**Objetivo deste Notebook:**
Executar a etapa de extração de dados brutos (*Raw Data*) prevista na metodologia. O script conecta-se à **YouTube Data API v3**, itera sobre uma lista de vídeos estratégicos selecionados e extrai todos os comentários disponíveis, contornando a limitação de paginação da API.

**Metodologia Aplicada:**
* **Fonte de Dados:** Comentários públicos em canais oficiais (OSINT).
* **Ferramenta:** Python (`google-api-python-client`).
* **Armazenamento:** Google Drive (Formato `.csv`).

In [3]:
# --- SCRIPT DE COLETA MASSIVA (ETL V2.1 - CORRIGIDO) ---
import os
import time
import pandas as pd
from googleapiclient.discovery import build
from google.colab import drive

# 1. MONTAGEM DO DRIVE
# Se já estiver montado, ele apenas avisa.
drive.mount('/content/drive')

# 2. CONFIGURAÇÕES
API_KEY = "INSIRA_SUA_CHAVE_AQUI"  # <--- RECOLOQUE SUA CHAVE AQUI
ALVOS_VIDEOS = [
    "sqS2JDhTOiQ",  # Enchentes RS
    "BY2hsjPervo",  # Vídeo 2 (exemplo)
    "s9vjDzPd29s"   # Vídeo 3 (exemplo)
]

def coletar_comentarios_video(youtube_service, video_id):
    """
    Função que baixa TODOS os comentários de um único vídeo.
    """
    comentarios = [] # <--- A lista nasce com nome em PORTUGUÊS
    print(f"--> Iniciando extração do vídeo: {video_id}")

    try:
        request = youtube_service.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            textFormat="plainText"
        )

        while request:
            response = request.execute()

            for item in response['items']:
                dado = item['snippet']['topLevelComment']['snippet']
                comentarios.append({
                    'Video_ID': video_id,
                    'Autor': dado.get('authorDisplayName', 'Desconhecido'),
                    'Data': dado.get('publishedAt'),
                    'Comentario': dado.get('textDisplay'),
                    'Likes': dado.get('likeCount', 0)
                })

            if 'nextPageToken' in response:
                request = youtube_service.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    pageToken=response['nextPageToken'],
                    maxResults=100,
                    textFormat="plainText"
                )
                time.sleep(0.5)
            else:
                break

        print(f"    [OK] {len(comentarios)} comentários extraídos.")
        return comentarios # <--- CORREÇÃO: Agora retorna a variável certa (antes estava 'comments')

    except Exception as e:
        print(f"    [ERRO] Falha no vídeo {video_id}: {e}")
        return []

# --- EXECUÇÃO PRINCIPAL ---
def main():
    print("--- INICIANDO OPERAÇÃO DE COLETA ---")

    youtube = build('youtube', 'v3', developerKey=API_KEY)
    todos_dados = []

    for video in ALVOS_VIDEOS:
        dados_video = coletar_comentarios_video(youtube, video)
        todos_dados.extend(dados_video)

    df_final = pd.DataFrame(todos_dados)

    if not df_final.empty:
        pasta_destino = '/content/drive/MyDrive/TCC_AMAN_2025'
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)

        caminho_arquivo = f'{pasta_destino}/base_dados_bruta.csv'
        df_final.to_csv(caminho_arquivo, index=False)

        print("\n" + "="*40)
        print(f"MISSÃO CUMPRIDA!")
        print(f"Total acumulado: {len(df_final)} comentários.")
        print(f"Arquivo salvo em: {caminho_arquivo}")
        print("="*40)
    else:
        print("\n[FALHA] Nenhum dado coletado.")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- INICIANDO OPERAÇÃO DE COLETA ---
--> Iniciando extração do vídeo: sqS2JDhTOiQ
    [OK] 301 comentários extraídos.
--> Iniciando extração do vídeo: BY2hsjPervo
    [OK] 164 comentários extraídos.
--> Iniciando extração do vídeo: s9vjDzPd29s
    [OK] 563 comentários extraídos.

MISSÃO CUMPRIDA!
Total acumulado: 1028 comentários.
Arquivo salvo em: /content/drive/MyDrive/TCC_AMAN_2025/base_dados_bruta.csv


## **Detalhamento da Arquitetura do Script**
O código desenvolvido acima implementa um pipeline de **ETL (Extract, Transform, Load)** robusto para a coleta de dados não estruturados (texto), seguindo as melhores práticas de Engenharia de Dados. Abaixo, detalha-se o funcionamento de cada módulo lógico:

### **A. Módulo de Extração (Extract)**
* **Conexão Segura:** Utilizamos a biblioteca `google-api-python-client` para estabelecer um túnel autenticado com a API do YouTube v3. A segurança é garantida via `API_KEY`.
* **Paginação Recursiva:** A API do YouTube possui uma limitação técnica que entrega apenas 100 comentários por requisição. Para contornar isso, implementamos um loop `while` que verifica a existência do token `nextPageToken`. Enquanto houver um token, o script continua solicitando a próxima página, garantindo a extração **exaustiva** (todos os comentários, não apenas os recentes).
* **Blindagem de Erros:** O bloco `try-except` protege a execução. Caso um vídeo específico tenha sido deletado ou tenha comentários desativados, o script registra o erro, mas não interrompe a coleta dos demais vídeos da lista.

### **B. Módulo de Transformação (Transform)**
* **Parsing de JSON:** A resposta bruta da API é um objeto JSON aninhado e complexo. O script navega cirurgicamente por essa estrutura (`item['snippet']['topLevelComment']...`) para extrair apenas os dados de interesse.
* **Normalização com `.get()`:** Utilizamos o método `.get()` nos dicionários para prevenir falhas (erros de chave). Se um campo (como 'likeCount') estiver ausente na fonte, o sistema preenche automaticamente com um valor padrão (0), garantindo a integridade tabular.
* **Estruturação Tabular:** Os dados, inicialmente em listas de dicionários, são convertidos para um **DataFrame do Pandas**. Essa estrutura facilita a manipulação matemática e estatística nas próximas etapas.

### **C. Módulo de Carga (Load)**
* **Persistência na Nuvem:** O resultado final não é mantido apenas na memória volátil (RAM). Ele é exportado fisicamente para o **Google Drive** em formato `.csv` (Comma Separated Values).
* **Versionamento de Dados:** O arquivo é salvo no diretório `TCC_AMAN_2025/base_dados_bruta.csv`, servindo como ponto de restauração (*checkpoint*). Isso permite que a etapa de Inteligência Artificial (Semana 2) seja iniciada sem a necessidade de reconectar à API do YouTube, economizando cota de requisição.

## **Próximos Passos**
Com a base de dados bruta salva, o próximo estágio do projeto consistirá em:
1.  **Limpeza de Texto (NLP):** Remoção de ruídos (emojis, links, quebras de linha).
2.  **Engenharia de Prompt:** Conexão com uma LLM (Gemini/GPT) para classificar o sentimento de cada comentário (Positivo/Negativo/Neutro).
3.  **Análise Exploratória:** Geração de estatísticas descritivas sobre a base rotulada.